In [26]:
import cv2
import mediapipe as mp
import numpy as np

import os
import pickle

# Preprocessing des datas 

In [27]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def get_landmarks(image_rgb, pose_model): 
    '''
    Récupère les landmarks d'une image donnée en RGB via un modèle déjà chargé.
    Return: un numpy array (33,4) ou None si rien n'est trouvé.
    '''

    results = pose_model.process(image_rgb)
    
    if results.pose_landmarks:

        pose_np = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark])
        return pose_np

    return None

In [28]:
def get_dataset(folder_path, path_save, pickle_name="dataset.pkl"): 
    ''' récupère les landmarks d'images dans un dossier via la fonction get_landmarks(image)

    folder_path : dossier où sont les images
    path_save   : dossier où sauvegarder le pickle
    pickle_name : nom du fichier pickle

    return: numpy array de shape (N, 132) si flatten (33*4)
    '''

    mp_pose = mp.solutions.pose
    os.makedirs(path_save, exist_ok=True)

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        data_list = []
        fichiers = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        fichiers.sort()

        path_save_pickle = os.path.join(path_save, pickle_name)

        for name in fichiers:
            img_path = os.path.join(folder_path, name)
            img = cv2.imread(img_path)

            if img is None:
                print(f"Impossible de lire : {img_path}")
                continue

            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            landmarks_vector = get_landmarks(img_rgb, pose)

            if landmarks_vector is not None:
                data_list.append(landmarks_vector.flatten())  # (132,)
            else:
                print(f"Ignorée (pas de pose détectée) : {name}")

        dataset = np.array(data_list, dtype=np.float32)

        with open(path_save_pickle, 'wb') as f:
            pickle.dump(dataset, f)

    return dataset

## attributs target  

In [29]:
def get_target(dataset, class_int): 
    ''''récupère la target du dataset'''

    nb_images = dataset.shape[0]
    class_id = class_int
    target = np.full(nb_images, class_id)

    return target 


# Build dataset et concatenisation en numpy array#

In [30]:
import os, json, pickle
import numpy as np
from collections import Counter

def build_full_dataset(data_root, path_save, poses=("cobra", "tree", "downdog","forwardfold","chair",
                                                   "warrior2","warrior3","plank","lotus")):
    """
    Construit X, y à partir de dossiers:
      data_root/cobra, data_root/tree, data_root/downdog

    Retour:
      X (N,132), y (N,)
    + sauvegarde un pickle par pose (comme vous faites déjà)
    """
    X_list = []
    y_list = []

    # mapping figé par l'ordre de `poses` pour être sur que les numéros des classes correspndent bien aux poses
    label2id = {pose_name: i for i, pose_name in enumerate(poses)}
    id2label = {i: pose_name for pose_name, i in label2id.items()}

    for pose_name in poses:
        class_id = label2id[pose_name]
        folder_path = os.path.join(data_root, pose_name)


        ds = get_dataset(
            folder_path=folder_path,
            path_save=path_save,
            pickle_name=f"dataset_{pose_name}.pkl"
        )

        target = get_target(ds, class_id)

        X_list.append(ds)
        y_list.append(target)

    X = np.vstack(X_list).astype(np.float32)
    y = np.concatenate(y_list).astype(np.int64)
    
    os.makedirs(path_save, exist_ok=True)

    # (optionnel) sauvegarde globale
    with open(os.path.join(path_save, "X_all.pkl"), "wb") as f:
        pickle.dump(X, f)
    with open(os.path.join(path_save, "y_all.pkl"), "wb") as f:
        pickle.dump(y, f)
    
    ## sauvegarde du mapping (facilité pour les appels pour la suite)
    mapping = {
        "labels": [id2label[i] for i in sorted(id2label)],  # ordre = sorties du modèle
        "label2id": label2id,
        "id2label": {str(k): v for k, v in id2label.items()}
    }
    
    with open(os.path.join(path_save, "labels_mapping.json"), "w", encoding="utf-8") as f:
        json.dump(mapping, f, ensure_ascii=False, indent=2)
        
        # sanity check : compte par classe réellement présente
    counts = Counter(y.tolist())
    print("\n Counts par id:", dict(counts))
    print("id 0 =", id2label.get(0), "| id 1 =", id2label.get(1),"| id 2 =", id2label.get(2),
          "| id 3 =", id2label.get(3), "| id 4 =", id2label.get(4),
          "| id 5 =", id2label.get(5), "| id 6 =", id2label.get(6), "| id 7 =", id2label.get(7),
          "|  id 8 =", id2label.get(8))
        
    
    return X, y

# Dataset des postures 

In [31]:
path_data = r'C:\Users\mvana\Documents\Formation data scientist\20. ACV\Posture_yoga\data'
path_save = "datasets"

X, y = build_full_dataset(path_data, path_save)
print(X.shape, y.shape)

Ignorée (pas de pose détectée) : 1_152.jpg
Ignorée (pas de pose détectée) : images69.jpg
Ignorée (pas de pose détectée) : 197.jpg
Ignorée (pas de pose détectée) : File49.jpg
Ignorée (pas de pose détectée) : PXL_20251217_103643021.MP.jpg
Ignorée (pas de pose détectée) : images145.jpg
Ignorée (pas de pose détectée) : downdog (20).jpg
Ignorée (pas de pose détectée) : downdog (20).png
Ignorée (pas de pose détectée) : downdog (24).jpg
Ignorée (pas de pose détectée) : downdog (78).jpg
Ignorée (pas de pose détectée) : File66.jpeg
Ignorée (pas de pose détectée) : File12.png
Ignorée (pas de pose détectée) : File17.png
Ignorée (pas de pose détectée) : File2.png
Ignorée (pas de pose détectée) : File27.png
Ignorée (pas de pose détectée) : File31.png
Ignorée (pas de pose détectée) : File4.png
Ignorée (pas de pose détectée) : File5.png
Ignorée (pas de pose détectée) : File62.png
Ignorée (pas de pose détectée) : File67.png
Ignorée (pas de pose détectée) : File79.jpeg
Ignorée (pas de pose détectée) : 

# Vérification de la sortie #

In [32]:
chemin_du_fichier = r'C:\Users\mvana\Documents\Formation data scientist\20. ACV\ACV_yoga\datasets\Y_all.pkl'

with open(chemin_du_fichier, 'rb') as f:
    dataset = pickle.load(f)
dataset.shape

(542,)

In [33]:
get_target(dataset, 1).shape

(542,)

In [34]:
dataset ### classe des images traitées dans y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,

## création des logos en numpy

In [38]:
labels = ["cobra","tree","downdog","forwardfold","chair","warrior2","warrior3","plank","child","lotus"]
logos_dir = r"C:\Users\mvana\Documents\Formation data scientist\20. ACV\Posture_yoga\data\logo"

import os
print(os.listdir(logos_dir))





['chair.png', 'child.png', 'cobra.png', 'downdog.png', 'forwardfold.png', 'lotus.jpg', 'plank.jpg', 'tree.jpg', 'warrior1.png', 'warrior2.jpg', 'warrior3.jpg']


In [39]:
import os
import numpy as np
import cv2

def load_logo_as_numpy(img_path, size=(128,128)):
    img_bgr = cv2.imread(img_path)
    if img_bgr is None:
        return None
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    if size is not None:
        img_rgb = cv2.resize(img_rgb, size)
    return img_rgb.astype(np.uint8)

def find_logo_file(logos_dir, label):
    exts = (".png", ".jpg", ".jpeg", ".webp")
    label_low = label.lower()

    candidates = []
    for f in os.listdir(logos_dir):
        f_low = f.lower()
        if f_low.endswith(exts):
            base = os.path.splitext(f_low)[0]  # nom sans extension
            # match exact OU "contient"
            if base == label_low or label_low in base:
                candidates.append(f)

    if len(candidates) == 0:
        return None

    # prend le 1er par ordre alphabétique (stable)
    candidates.sort()
    return os.path.join(logos_dir, candidates[0])

def save_logos_npz(logos_dir, labels, out_npz_path, size=(128,128)):
    arrays = {}
    chosen_files = {}

    for label in labels:
        img_path = find_logo_file(logos_dir, label)
        if img_path is None:
            print(f"[WARN] Logo manquant pour '{label}'")
            continue

        arr = load_logo_as_numpy(img_path, size=size)
        if arr is None:
            print(f"[WARN] Impossible de lire: {img_path}")
            continue

        arrays[label] = arr
        chosen_files[label] = os.path.basename(img_path)

    np.savez_compressed(out_npz_path, **arrays)
    return list(arrays.keys()), chosen_files


In [40]:
import json
import os

labels = ["cobra","tree","downdog","forwardfold","chair","warrior2","warrior3","plank","child","lotus"]
logos_dir = r"C:\Users\mvana\Documents\Formation data scientist\20. ACV\Posture_yoga\data\logo"

out_dir = "artifacts"
os.makedirs(out_dir, exist_ok=True)
npz_path = os.path.join(out_dir, "logos.npz")

kept, chosen_files = save_logos_npz(logos_dir, labels, npz_path, size=(128,128))

with open("yoga_config.json", "r", encoding="utf-8") as f:
    cfg = json.load(f)

cfg["logos_npz_path"] = npz_path.replace("\\", "/")
cfg["logos_npz_keys"] = kept
cfg["logos_files_used"] = chosen_files  # pratique pour debug

with open("yoga_config.json", "w", encoding="utf-8") as f:
    json.dump(cfg, f, ensure_ascii=False, indent=2)

print("✅ logos.npz créé:", npz_path)
print("✅ logos trouvés:", kept)


✅ logos.npz créé: artifacts\logos.npz
✅ logos trouvés: ['cobra', 'tree', 'downdog', 'forwardfold', 'chair', 'warrior2', 'warrior3', 'plank', 'child', 'lotus']
